In [3]:
using Pkg
Pkg.activate("../.")
Pkg.instantiate()

  Activating project at `~/Desktop/research/rfq-nn`
┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.9.1). Unexpected behavior may occur.
└ @ ~/Desktop/research/rfq-nn/Manifest.toml:0


In [2]:
include("../scan_hyperparameters_withcellnum_functions.jl")
include("../helpers_temp.jl")

generatemodelid (generic function with 1 method)

In [17]:
target_directory = "../data/full_with_cellnumber/"

x_raw_df, y_df, cellnumber_df = getrawdata_withcellnum(target_directory)
x_raw_df = hcat(x_raw_df, cellnumber_df .% 2)

println("Done!")

Decorrelating...
Done!


In [32]:
path = "../indexes"
train_idx_df = CSV.File("$path/train_indexes.csv"; header=0) |> DataFrame
test_idx_df = CSV.File("$path/test_indexes.csv"; header=0) |> DataFrame
train_idx = vec(Matrix(train_idx_df));
test_idx = vec(Matrix(test_idx_df));

In [45]:
@assert length(train_idx) + length(test_idx) <= size(x_df)[1]

In [46]:
# mask for transmission
transmission_mask = findall(x -> 60 <= x <= 120, y_df[:, "OBJ1"]);

In [74]:
# new train and test indexes
train_idx_with_cut = intersect(train_idx, transmission_mask);
test_idx_with_cut = intersect(test_idx, transmission_mask);

In [76]:
# sanity checking
@assert all([idx ∈ train_idx for idx in train_idx_with_cut])
@assert all([idx ∈ test_idx for idx in test_idx_with_cut])

@assert all([idx ∈ transmission_mask for idx in train_idx_with_cut])
@assert all([idx ∈ transmission_mask for idx in test_idx_with_cut])

In [77]:
length(train_idx_with_cut) / (length(train_idx_with_cut) + length(test_idx_with_cut))

0.799067277717749

Still at 80% train / 20% test split. Nice!

In [78]:
length(train_idx_with_cut)

128677

In [79]:
CSV.write(
    "$path/transmission_above_60/train_indexes.csv",
    DataFrame(Dict("col1" => train_idx_with_cut)); writeheader=false
)

CSV.write(
    "$path/transmission_above_60/test_indexes.csv",
    DataFrame(Dict("col1" => test_idx_with_cut)); writeheader=false
)

"../indexes/transmission_above_60/test_indexes.csv"